In [1]:
# Restarta python caso necessário.
# from databricks.sdk.runtime import *
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp


pd.set_option("display.max_columns", None)

### Testes Modulares - Referente ao tópico 3 - Desenho de Execução do Processo (Modelo de Priorização de Score e GH)

### Geração de Massa

In [3]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({
                             'solicitante.numeroDocumento':['123456789'],
                             'solicitante.dataAdmissaoFopag':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistroPrimeiroContrato':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistro':["2002-11-05T17:18:43.888Z"],
                             'solicitante.flagMAT30d':[0],
                             'solicitante.flagMAT35d':[0],
                             'solicitante.flagMAT60d':[0],
                             'solicitante.flagMAT90d':[0],
                             'solicitante.fonteRenda':['XYZ'],
                             'solicitante.flagPortabilidade':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'solicitante.consumerID':[12345678987654321],
                             'solicitante.investimentoTotal':[0],
                             'solicitante.flagInvestimentoPrincipalidade':[0],
                             'solicitante.flagPrincipalidadeAll':[0],
                             'solicitante.flagPrincipalidade':[0],
                             'solicitante.mesFlagPrincipalidade':[0],
                             'solicitante.flagAllowlist':["-99999"],
                             'solicitante.flagBlocklist':["-99999"],
                             'solicitante.flagCartaoCreditoContratado':[0],
                             'solicitante.flagLimiteGarantidoContratado':[0],
                             
            
                             'solicitante.ghAppcardsBlendInternoOnline':["missing"],
                             'solicitante.scoreAppcardsBlendInternoOnline':[0],
                             'solicitante.ghAppcardsBlendExternoOnline':["missing"],
                             'solicitante.scoreAppcardsBlendExternoOnline':[0],
                             'solicitante.ghBlendIntExt':["missing"],
                             'solicitante.scoreBlendIntExt':[0],
                             'solicitante.ghSerasa':["missing"],
                             'solicitante.scoreSerasaAl':[0],
                             
                        
                             'solicitante.blendFinalAl':[500],
                             'solicitante.ghFinalScr':["B"],
                             'solicitante.origemBlend':["04.SERASA"]
    
                            })

In [4]:
#Criação das massas com casos forçados


# Criacao Modelos de Priorização - ADICIONADO NOVA VERSAO

massa_modelosPriorizacao = sherlock.Gera_Massa(base_entrada)

massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendInternoOnline',["-99999",'B','missing','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendInternoOnline',[-99999,'650',-1,0])
massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendExternoOnline',["-99999",'D','missing','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendExternoOnline',[-99999,500,-1])
massa_modelosPriorizacao.gera_cenario('solicitante.ghBlendIntExt',['E','missing','0',"-99999"])   #passar -99999 aqui dps
massa_modelosPriorizacao.gera_cenario('solicitante.scoreBlendIntExt',[100,-1,-99999])
massa_modelosPriorizacao.gera_cenario('solicitante.ghSerasa',['F','missing','0',"-99999"])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreSerasaAl',[1000,0,-1,-99999])

massa_modelosPriorizacao.gera_cenario('solicitante.blendFinalAl',[1000,-500])
massa_modelosPriorizacao.gera_cenario('solicitante.ghFinalScr',["B","missing"])
massa_modelosPriorizacao.gera_cenario('solicitante.origemBlend',["04.SERASA","missing"])

# massa_modelosPriorizacao = massa_modelosPriorizacao.get_massa_final()
massa_modelosPriorizacao.set_bom_arqv()
massa_modelosPriorizacao.converte_massa()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
listaResultado = rp.execucaoModular(massa_modelosPriorizacao.get_massa_json(),rp.CriacaoModelosPriorizacao)

In [6]:
# Renomeação das colunas para o cabeçalho do RMA
massa_modelosPriorizacao.rename_cabecalho()

In [7]:
#Contagem da qtd de regras negativas em que o cliente se enquadrou
tam = 0
for x in listaResultado:
    t = len(x['payloadHomol']['saidas']['regrasNegativas'])
    x['payloadHomol']['saidas']['qtdNegativas'] = t
    if t > tam:
        tam = t

print("tam: "+str(tam))

for y in range (0,tam):
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".nomeRegra","regrasNegativas#"+str(y+1)+".nomeRegra")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".descricao","regrasNegativas#"+str(y+1)+".descRegra")
    
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".nome","listaLogs#"+str(y+1)+".nome")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".valor","listaLogs#"+str(y+1)+".valor")




#saidas

massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.ghFinal','ghFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.scoreFinal','scoreFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.origemModeloApplication','origemModeloApplication')



massa_modelosPriorizacao.get_massa_dm()



tam: 0


,Data-ID,Result-State,Result-Detail,numeroDocumento,dataAdmissaoFopag,dataRegistroPrimeiroContrato,dataRegistro,flagMAT30d,flagMAT35d,flagMAT60d,flagMAT90d,fonteRenda,flagPortabilidade,rendaLiquidaPicpay,consumerID,investimentoTotal,flagInvestimentoPrincipalidade,payloadParam.solicitante.flagPrincipalidadeAll,flagPrincipalidade,mesFlagPrincipalidade,flagAllowlist,flagBlocklist,flagCartaoCreditoContratado,flagLimiteGarantidoContratado,ghAppcardsBlendInternoOnline,scoreAppcardsBlendInternoOnline,ghAppcardsBlendExternoOnline,scoreAppcardsBlendExternoOnline,ghBlendIntExt,scoreBlendIntExt,ghSerasa,scoreSerasaAl,blendFinalAl,ghFinalScr,origemBlend,result:ghFinal,expected:ghFinal,result:scoreFinal,expected:scoreFinal,result:origemModeloApplication,expected:origemModeloApplication
0,1,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,-99999,-99999,-99999,-99999,E,100,F,1000,1000,B,04.SERASA,,B,,1000,,04.SERASA
1,2,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,-99999,-99999,-99999,-99999,E,100,F,1000,1000,B,missing,,B,,1000,,missing
2,3,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,-99999,-99999,-99999,-99999,E,100,F,1000,1000,missing,04.SERASA,,missing,,1000,,04.SERASA
3,4,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,-99999,-99999,-99999,-99999,E,100,F,1000,1000,missing,missing,,missing,,1000,,missing
4,5,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,-99999,-99999,-99999,-99999,E,100,F,1000,-500,B,04.SERASA,,B,,-500,,04.SERASA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294907,294908,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,0,0,0,-1,-99999,-99999,-99999,-99999,1000,missing,missing,,missing,,0,,Interno + Serasa + SCR
294908,294909,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,0,0,0,-1,-99999,-99999,-99999,-99999,-500,B,04.SERASA,,missing,,0,,Interno + Serasa + SCR
294909,294910,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,0,0,0,-1,-99999,-99999,-99999,-99999,-500,B,missing,,missing,,0,,Interno + Serasa + SCR
294910,294911,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,0,0,0,-1,-99999,-99999,-99999,-99999,-500,missing,04.SERASA,,missing,,0,,Interno + Serasa + SCR


### Conversão da massa para CSV

In [8]:
massa_modelosPriorizacao.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaModelosPriorizacaoV2.csv',index=False)

### Quebra da massa em pedaços

In [9]:

dataframeGrande = massa_modelosPriorizacao.get_massa_dm()
# dataframeGrande

num_dataframes = 20

split_dataframes = np.array_split(dataframeGrande, num_dataframes)

# i=6
for i, df in enumerate(split_dataframes):
    df.to_csv(f'Massas/Modulares/massaModelosPriorizacaoV2-part{i+1}.csv', index=False)

c:\Users\T51657\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


#### Análises

In [10]:
dataID = 1
print(json.dumps(listaResultado[dataID-1],indent=4))

{
    "marcacaoMesa": "",
    "etapa": "",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "idade": 30,
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "REGULAR",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 10000,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
    

In [11]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraGhFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,-1,211968
1,1,27648
2,2,13824
3,3,13824
4,4,6912
5,5,20736


In [12]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraScoreFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,-1,211968
1,1,55296
2,2,9216
3,3,9216
4,4,4608
5,5,4608


## testes casos especificos 

In [13]:
#Criação das massas com casos forçados


# Criacao Modelos de Priorização - ADICIONADO NOVA VERSAO

massa_modelosPriorizacao = sherlock.Gera_Massa(base_entrada)

massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendInternoOnline',["B"])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendInternoOnline',[-1])
massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendExternoOnline',['D'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendExternoOnline',[500])
massa_modelosPriorizacao.gera_cenario('solicitante.ghBlendIntExt',['E'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreBlendIntExt',[-1])
massa_modelosPriorizacao.gera_cenario('solicitante.ghSerasa',['missing'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreSerasaAl',[1000])

massa_modelosPriorizacao.gera_cenario('solicitante.blendFinalAl',[1000])
massa_modelosPriorizacao.gera_cenario('solicitante.ghFinalScr',["B"])
massa_modelosPriorizacao.gera_cenario('solicitante.origemBlend',["04.SERASA"])

# massa_modelosPriorizacao = massa_modelosPriorizacao.get_massa_final()
massa_modelosPriorizacao.set_bom_arqv()
massa_modelosPriorizacao.converte_massa()

listaResultado = rp.execucaoModular(massa_modelosPriorizacao.get_massa_json(),rp.CriacaoModelosPriorizacao)

# Renomeação das colunas para o cabeçalho do RMA
massa_modelosPriorizacao.rename_cabecalho()

#Contagem da qtd de regras negativas em que o cliente se enquadrou
tam = 0
for x in listaResultado:
    t = len(x['payloadHomol']['saidas']['regrasNegativas'])
    x['payloadHomol']['saidas']['qtdNegativas'] = t
    if t > tam:
        tam = t

print("tam: "+str(tam))

for y in range (0,tam):
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".nomeRegra","regrasNegativas#"+str(y+1)+".nomeRegra")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".descricao","regrasNegativas#"+str(y+1)+".descRegra")
    
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".nome","listaLogs#"+str(y+1)+".nome")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".valor","listaLogs#"+str(y+1)+".valor")




#saidas

massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.ghFinal','ghFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.scoreFinal','scoreFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.origemModeloApplication','origemModeloApplication')



massa_modelosPriorizacao.get_massa_dm()



--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso
tam: 0


,Data-ID,Result-State,Result-Detail,numeroDocumento,dataAdmissaoFopag,dataRegistroPrimeiroContrato,dataRegistro,flagMAT30d,flagMAT35d,flagMAT60d,flagMAT90d,fonteRenda,flagPortabilidade,rendaLiquidaPicpay,consumerID,investimentoTotal,flagInvestimentoPrincipalidade,payloadParam.solicitante.flagPrincipalidadeAll,flagPrincipalidade,mesFlagPrincipalidade,flagAllowlist,flagBlocklist,flagCartaoCreditoContratado,flagLimiteGarantidoContratado,ghAppcardsBlendInternoOnline,scoreAppcardsBlendInternoOnline,ghAppcardsBlendExternoOnline,scoreAppcardsBlendExternoOnline,ghBlendIntExt,scoreBlendIntExt,ghSerasa,scoreSerasaAl,blendFinalAl,ghFinalScr,origemBlend,result:ghFinal,expected:ghFinal,result:scoreFinal,expected:scoreFinal,result:origemModeloApplication,expected:origemModeloApplication
0,1,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,0,0,0,0,0,-99999,-99999,0,0,B,-1,D,500,E,-1,missing,1000,1000,B,04.SERASA,,B,,500,,Serasa + SCR
